## Scraping 
When using this scraper, you may get rate limited. It is recommended to start at a checkpoint by using the `Start at a course / unit` section. To save a checkpoint see the Cleaning section below.

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import time
import csv
import glob

In [3]:
# Start at a course / unit
no_checkpoint = False # Set true or false depending on whether you are starting at a certain course
start_unit = "Sociology"
start_course = "SOCI 105 - Ethnographic Film: Media Meth"

driver = webdriver.Chrome()

# Load the webpage where you want to use the cookies
driver.get("http://academicaffairs.ucsd.edu/")

# Load cookies from the JSON file
with open('academicaffairs.ucsd.edu.cookies.json', 'r') as file:
    cookies_raw = json.load(file)

# Add each cookie to the Selenium
for cookie in cookies_raw:
    driver.add_cookie(cookie)

# Refresh the page to apply the cookies
driver.refresh()

# Get SET Page
driver.get("http://academicaffairs.ucsd.edu/Modules/Evals/SET/Reports/Search.aspx")

time.sleep(5)

# Find the unit dropdown and collect all unit options
unit_dropdown = driver.find_element(By.ID, "ContentPlaceHolder1_EvalsContentPlaceHolder_ddlUnit")
unit_options = unit_dropdown.find_elements(By.TAG_NAME, "option")
units = [(option.text, option.get_attribute("value")) for option in unit_options if option.get_attribute("value")]
with open('scraped_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Instructor', 'Course', 'Term', 'Enrolled/Resp Rate', 'Avg Grade Received', 'Avg Hours Worked', 'Student Learning', 'Course Structure', 'Class Environment'])

    for unit in units:
        # Skip courses until the start unit is found
        if not found_start_unit and not no_start:
            if unit[0] == start_unit:
                found_start_unit = True
            else:
                continue
        # Select a unit
        unit_dropdown = driver.find_element(By.ID, "ContentPlaceHolder1_EvalsContentPlaceHolder_ddlUnit")
        for option in unit_dropdown.find_elements(By.TAG_NAME, "option"):
            if option.get_attribute("value") == unit[1]:
                option.click()
                break
    
        # Wait for the course dropdown to be populated
        time.sleep(2)
        course_dropdown = driver.find_element(By.ID, "ContentPlaceHolder1_EvalsContentPlaceHolder_ddlCourse")
        course_options = course_dropdown.find_elements(By.TAG_NAME, "option")
    
        # Extract course information
        courses = [(option.text, option.get_attribute("value")) for option in course_options if option.get_attribute("value")]
    
        for course in courses:
            # Skip courses until the start course is found
            if not found_start_course and not no_start:
                if course[0] == start_course:
                    found_start_course = True
                else:
                    continue

            # Select a course
            course_dropdown = driver.find_element(By.ID, "ContentPlaceHolder1_EvalsContentPlaceHolder_ddlCourse")
            for option in course_dropdown.find_elements(By.TAG_NAME, "option"):
                if option.get_attribute("value") == course[1]:
                    option.click()
                    break
    
            # Click the search button
            search_button = driver.find_element(By.ID, "ContentPlaceHolder1_EvalsContentPlaceHolder_btnSubmit")
            search_button.click()
    
            # Wait for the table to load
            time.sleep(5)
    
            # Scrape the table data using BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # Try to find a table on the page
            try:
                # Scrape the First Table (which should be SET or sometimes CAPES if there are no SET results)
                table = soup.find("table")
            
                # Extract table rows
                rows = table.find('tbody').find_all('tr')
                for row in rows:
                    cells = row.find_all('td')
                    row_data = [cell.text.strip() for cell in cells]
                    writer.writerow([unit[0], course[0]] + row_data)
            except:
                continue

# Close the browser
driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7BCCF22C2+60002]
	(No symbol) [0x00007FF7BCC6CA59]
	(No symbol) [0x00007FF7BCB27EDA]
	(No symbol) [0x00007FF7BCAFD5B5]
	(No symbol) [0x00007FF7BCBA3727]
	(No symbol) [0x00007FF7BCBBB3A1]
	(No symbol) [0x00007FF7BCB9C033]
	(No symbol) [0x00007FF7BCB69657]
	(No symbol) [0x00007FF7BCB6A251]
	GetHandleVerifier [0x00007FF7BD003E2D+3278285]
	GetHandleVerifier [0x00007FF7BD050190+3590448]
	GetHandleVerifier [0x00007FF7BD0461D0+3549552]
	GetHandleVerifier [0x00007FF7BCDA1DE6+779654]
	(No symbol) [0x00007FF7BCC77ACF]
	(No symbol) [0x00007FF7BCC72EE4]
	(No symbol) [0x00007FF7BCC73072]
	(No symbol) [0x00007FF7BCC62C4F]
	BaseThreadInitThunk [0x00007FF9B2247344+20]
	RtlUserThreadStart [0x00007FF9B3ADCC91+33]


## Cleaning Data &#x1F6AE;
After scraping the data, there are some things to clean.
#### Todo List
- Fix types
- Handle null values
- Column Names
    - Implement naming conventions.
- Avg Grade Received Column
    - Remove trailing white space and commas
- Course Column and Indexes
    - Remove redundancy
    - Group by instructor, course and term
- Enrolled/Resp Rate
    - Separate Columns
- Term Column
    - Make sure terms start from Summer 2023 (when SET was introduced)
- Creating and saving checkpoints as needed
- Combining the checkpoints after finishing entire SET

In [6]:
scraped_data = pd.read_csv('scraped_data.csv')
scraped_data.head()

Instructor  \
Sociology SOCI 105 - Ethnographic Film: Media Meth   Jules-Rosette, Bennetta W.   
          SOCI 106 - Comparative&Historical Methods       Payne, Christine Anna   
          SOCI 106 - Comparative&Historical Methods        Ahn, Hyangseon Irene   
          SOCI 109 - Analysis of Sociological Data           Mackenna, Bernardo   
          SOCI 110 - Qualitative Res/Educ Settings   Holkenbrink-Monk, Charlene   

                                                                                              Course  \
Sociology SOCI 105 - Ethnographic Film: Media Meth    SOCI 105 - Ethnographic Film: Media Meth (A00)   
          SOCI 106 - Comparative&Historical Methods  SOCI 106 - Comparative&Historical Methods (A00)   
          SOCI 106 - Comparative&Historical Methods  SOCI 106 - Comparative&Historical Methods (A00)   
          SOCI 109 - Analysis of Sociological Data    SOCI 109 - Analysis of Sociological Data (A00)   
          SOCI 110 - Qualitative Res/Educ Settings    SOCI 110 - Qualitative Res/Educ Settings (A00)   

                                                     Term Enrolled/Resp Rate  \
Sociology SOCI 105 - Ethnographic Film: Media Meth   FA23          5(80.00%)   
          SOCI 106 - Comparative&Historical Methods  SP24         28(28.57%)   
          SOCI 106 - Comparative&Historical Methods  WI24         25(28.00%)   
          SOCI 109 - Analysis of Sociological Data   S123         17(23.53%)   
          SOCI 110 - Qualitative Res/Educ Settings   FA23         22(36.36%)   

                                                                                    Avg Grade Received  \
Sociology SOCI 105 - Ethnographic Film: Media Meth   3.93                                          ...   
          SOCI 106 - Comparative&Historical Methods  3.40                                          ...   
          SOCI 106 - Comparative&Historical Methods  3.12                                          ...   
          SOCI 109 - Analysis of Sociological Data   2.87                                          ...   
          SOCI 110 - Qualitative Res/Educ Settings   3.18                                          ...   

                                                    Avg Hours Worked  \
Sociology SOCI 105 - Ethnographic Film: Media Meth             10.00   
          SOCI 106 - Comparative&Historical Methods             6.00   
          SOCI 106 - Comparative&Historical Methods             5.00   
          SOCI 109 - Analysis of Sociological Data              7.50   
          SOCI 110 - Qualitative Res/Educ Settings              6.88   

                                                     Student Learning  \
Sociology SOCI 105 - Ethnographic Film: Media Meth               4.56   
          SOCI 106 - Comparative&Historical Methods              4.69   
          SOCI 106 - Comparative&Historical Methods              3.32   
          SOCI 109 - Analysis of Sociological Data               3.19   
          SOCI 110 - Qualitative Res/Educ Settings               4.88   

                                                     Course Structure  \
Sociology SOCI 105 - Ethnographic Film: Media Meth               4.56   
          SOCI 106 - Comparative&Historical Methods              4.68   
          SOCI 106 - Comparative&Historical Methods              3.25   
          SOCI 109 - Analysis of Sociological Data               3.50   
          SOCI 110 - Qualitative Res/Educ Settings               4.81   

                                                     Class Environment  
Sociology SOCI 105 - Ethnographic Film: Media Meth                4.50  
          SOCI 106 - Comparative&Historical Methods               4.72  
          SOCI 106 - Comparative&Historical Methods               4.07  
          SOCI 109 - Analysis of Sociological Data                4.50  
          SOCI 110 - Qualitative Res/Educ Settings                4.91

In [92]:
scraped_data.dtypes

Instructor             object
Course                 object
Term                   object
Enrolled/Resp Rate     object
Avg Grade Received     object
Avg Hours Worked       object
Student Learning      float64
Course Structure      float64
Class Environment     float64
dtype: object

In [94]:
scraped_data[[col for col in scraped_data.columns]].isnull().sum()

Instructor              0
Course                  0
Term                    0
Enrolled/Resp Rate      6
Avg Grade Received      0
Avg Hours Worked      170
Student Learning        2
Course Structure      255
Class Environment     256
dtype: int64

In [12]:
df = scraped_data.reset_index()

# Handle nulls
df = df.fillna(-1)

# Separate Enrolled/Resp Rate Column
sep = df['Enrolled/Resp Rate'].str.extract(r'(\d+)\((\d+\.\d+)%\)')
sep = sep.fillna(-1)
df['enrolled'] = sep[0].astype(int)
df['resp_rate'] = sep[1].astype(float)
df['resp_rate'] *= 0.01

# Drop irrelevant columns
df = df.drop(['level_0', 'level_1', 'Enrolled/Resp Rate'], axis = 1)

# Column Names
new_columns = ['instructor', 'course', 'term', 'avg_grade', 'avg_hours', 'student_learning_rating', 'course_structure_rating', 'class_environment_rating', 'num_enrolled', 'resp_rate']
df.columns = new_columns

# Clean course column
df['course'] = df['course'].str.replace(r"\ \(\d{3}\)", "", regex=True)

# Clean avg_grade columns
df['avg_grade'] = df['avg_grade'].str.extract(r"(\d\.\d+)")
df['avg_grade'] = df['avg_grade'].astype('float')


df.head()


,instructor,course,term,avg_grade,avg_hours,student_learning_rating,course_structure_rating,class_environment_rating,num_enrolled,resp_rate
0,"Jules-Rosette, Bennetta W.",SOCI 105 - Ethnographic Film: Media Meth (A00),FA23,3.93,10.00,4.56,4.56,4.50,5,0.8000
1,"Payne, Christine Anna",SOCI 106 - Comparative&Historical Methods (A00),SP24,3.40,6.00,4.69,4.68,4.72,28,0.2857
2,"Ahn, Hyangseon Irene",SOCI 106 - Comparative&Historical Methods (A00),WI24,3.12,5.00,3.32,3.25,4.07,25,0.2800
3,"Mackenna, Bernardo",SOCI 109 - Analysis of Sociological Data (A00),S123,2.87,7.50,3.19,3.50,4.50,17,0.2353
4,"Holkenbrink-Monk, Charlene",SOCI 110 - Qualitative Res/Educ Settings (A00),FA23,3.18,6.88,4.88,4.81,4.91,22,0.3636


In [13]:
df.dtypes

instructor                   object
course                       object
term                         object
avg_grade                   float64
avg_hours                    object
student_learning_rating     float64
course_structure_rating     float64
class_environment_rating    float64
num_enrolled                  int64
resp_rate                   float64
dtype: object

### Save the cleaned checkpoint

In [ ]:
df.to_csv("checkpoint_name")

### Combining the checkpoints

In [18]:
csv_files = glob.glob("checkpoint_*.csv")

In [19]:
dfs = [pd.read_csv(file) for file in csv_files]

In [29]:
df = pd.concat(dfs, ignore_index=True)
df = df.iloc[:, 1:]

In [30]:
df.describe()

,avg_grade,student_learning_rating,course_structure_rating,class_environment_rating,num_enrolled,resp_rate
count,5090.000000,6401.000000,6401.00000,6401.000000,6401.000000,6401.000000
mean,3.418063,4.696329,3.38957,3.560803,61.253242,0.371648
std,0.444344,1.642014,2.07035,2.132852,82.708634,0.278624
min,1.380000,-1.000000,-1.00000,-1.000000,-1.000000,-0.010000
25%,3.120000,4.180000,3.74000,4.180000,14.000000,0.187500
50%,3.490000,4.490000,4.29000,4.500000,28.000000,0.325000
75%,3.800000,4.750000,4.58000,4.700000,74.000000,0.571400
max,4.000000,19.170000,5.00000,5.000000,613.000000,1.861100


In [33]:
df['term'].value_counts()[:10]

term
WI24    1575
SP24    1490
FA23    1455
S123     332
S223     283
SP23     175
WI17      42
SP19      35
WI20      33
WI11      32
Name: count, dtype: int64

In [36]:
df = df[df['term'].isin(["S223", "S123", "FA23", "SP24", "WI24"])] # include the SET only data

In [37]:
df.head()

,instructor,course,term,avg_grade,avg_hours,student_learning_rating,course_structure_rating,class_environment_rating,num_enrolled,resp_rate
0,"Given, William Allan",AWP 3 - Analytical Writing,SP24,2.37,7.29,4.36,4.43,4.54,10,0.7000
1,"Kolodezh, Samuel",AWP 3 - Analytical Writing,SP24,1.86,8.33,4.67,4.58,4.42,7,0.4286
2,"Gilbert, Peter W",AWP 3 - Analytical Writing,SP24,1.81,8.00,5.00,5.00,4.88,7,0.5714
3,"Gilbert, Peter W",AWP 3 - Analytical Writing,WI24,2.77,8.60,4.20,3.90,4.30,7,0.7143
4,"Gilbert, Peter W",AWP 3 - Analytical Writing,WI24,3.13,6.25,4.41,4.47,4.47,8,1.0000


In [ ]:
df.to_csv("